<a href="https://colab.research.google.com/github/smulani80/LangChainQueryCSV/blob/main/LangChain_Query_csv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchainhub langchain-openai chromadb

  Using cached opentelemetry_sdk-1.22.0-py3-none-any.whl (105 kB)
  Using cached opentelemetry_instrumentation_asgi-0.43b0-py3-none-any.whl (14 kB)
  Using cached opentelemetry_instrumentation-0.43b0-py3-none-any.whl (28 kB)
  Using cached opentelemetry_semantic_conventions-0.43b0-py3-none-any.whl (36 kB)
  Using cached opentelemetry_util_http-0.43b0-py3-none-any.whl (6.9 kB)
  Using cached opentelemetry_api-1.22.0-py3-none-any.whl (57 kB)
  Attempting uninstall: opentelemetry-util-http
    Found existing installation: opentelemetry-util-http 0.41b0
    Uninstalling opentelemetry-util-http-0.41b0:
      Successfully uninstalled opentelemetry-util-http-0.41b0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.41b0
    Uninstalling opentelemetry-semantic-conventions-0.41b0:
      Successfully uninstalled opentelemetry-semantic-conventions-0.41b0
  Attempting uninstall: opentelemetry-api
    Found existing insta

In [2]:
!pip install sentence-transformers

In [3]:
!pip install openllm

  Using cached build-0.10.0-py3-none-any.whl (17 kB)
  Using cached opentelemetry_api-1.20.0-py3-none-any.whl (57 kB)
  Using cached opentelemetry_instrumentation_asgi-0.41b0-py3-none-any.whl (13 kB)
  Using cached opentelemetry_instrumentation-0.41b0-py3-none-any.whl (25 kB)
  Using cached opentelemetry_sdk-1.20.0-py3-none-any.whl (103 kB)
  Using cached opentelemetry_semantic_conventions-0.41b0-py3-none-any.whl (26 kB)
  Using cached opentelemetry_util_http-0.41b0-py3-none-any.whl (6.9 kB)
  Attempting uninstall: opentelemetry-util-http
    Found existing installation: opentelemetry-util-http 0.43b0
    Uninstalling opentelemetry-util-http-0.43b0:
      Successfully uninstalled opentelemetry-util-http-0.43b0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.43b0
    Uninstalling opentelemetry-semantic-conventions-0.43b0:
      Successfully uninstalled opentelemetry-semantic-conventions-0.43b0
  Attempting 

In [4]:
!pip install huggingface_hub

In [5]:
# !pip uninstall lark
!pip install lark

In [6]:
!pip install environ python-environ

In [40]:
import environ
env = environ.Env()
environ.Env.read_env()
# API_KEY = env('OPENAI_API_KEY')
HF_API_KEY = env('HUGGINGFACEHUB_API_TOKEN')
#print(API_KEY)
#print(HF_API_KEY)

In [8]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [9]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='/content/sample_data/Movie_Review_Dataset.csv' , source_column="Rating")
data = loader.load()

In [10]:
print(data), len(data)

[Document(page_content="\ufeffSrNo: 1\nMovieName: 2012\nRating: 9\nPolarity: 1\nComments: I have to say this movie is very tense. The disasters in it make you think, will the world really end this way? It's one of those films where it gets your mind thinking about the world around us and how natural disasters can and will happen.\nThe reason I give 2012 a 9 is it is a film where it makes you think, it makes you think about being prepared for disasters great or small, basically it's a movie about reality but its fiction. Don't get me wrong it was a great movie, a real edge of your seat type of deal. For one the natural disasters in the film can happen anytime, for example, the yellow stone caldera is a real super volcano that scientists and geologist have been studying. They do say it will erupt again in the future, but they don't know when? as for the earthquakes those are real, the shifting of continents? I guess a major event like that can happen? It did millions of years ago.\nI for

(None, 50)

In [11]:
print(data[0:2])

[Document(page_content="\ufeffSrNo: 1\nMovieName: 2012\nRating: 9\nPolarity: 1\nComments: I have to say this movie is very tense. The disasters in it make you think, will the world really end this way? It's one of those films where it gets your mind thinking about the world around us and how natural disasters can and will happen.\nThe reason I give 2012 a 9 is it is a film where it makes you think, it makes you think about being prepared for disasters great or small, basically it's a movie about reality but its fiction. Don't get me wrong it was a great movie, a real edge of your seat type of deal. For one the natural disasters in the film can happen anytime, for example, the yellow stone caldera is a real super volcano that scientists and geologist have been studying. They do say it will erupt again in the future, but they don't know when? as for the earthquakes those are real, the shifting of continents? I guess a major event like that can happen? It did millions of years ago.\nI for

In [12]:
#splits
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=0)

In [13]:
%%time
split_docs = splitter.split_documents(data)
len(split_docs)

CPU times: user 36.6 ms, sys: 0 ns, total: 36.6 ms
Wall time: 37.9 ms


741

In [14]:
##Configure our embedding model
from transformers import AutoConfig
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
embedding_model_id = "BAAI/bge-small-en-v1.5"

config = AutoConfig.from_pretrained(embedding_model_id)
embedding_dimensions = config.hidden_size

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/93.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# Store splits
from langchain.vectorstores import Chroma

persist_directory = '/content/sample_data/LangChainTest/chroma_vectordb_dir1/'

# Create the vector store
chroma_vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

print(chroma_vectordb._collection.count())


741


In [17]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [ ]:
# # Use a filter to only retrieve documents from a specific user
# retriever = db.as_retriever(
#     search_kwargs={'filter': {'user_id':'user1'}}
# )

In [37]:
from langchain.prompts import PromptTemplate
#use standard defined prompt
prompt1 = hub.pull("rlm/rag-prompt")

# Build customized prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Start answer with apology statement. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
prompt2 = PromptTemplate.from_template(template)


In [30]:
print(prompt1)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [38]:
print(prompt2)

input_variables=['context', 'question'] template='Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Start answer with apology statement. Always say "thanks for asking!" at the end of the answer. \n{context}\nQuestion: {question}\nHelpful Answer:'


In [33]:
# retriever=chroma_vectordb.as_retriever()
retriever=chroma_vectordb.as_retriever(search_kwargs={'filter': {'source': '1'}})  #Filter only negative comments of rating 1

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [35]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt1
    | llm
    | StrOutputParser()
)

generated_answer = rag_chain.invoke("Why audience didn't like the movie 2012 ?")
print(generated_answer)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Why audience didn't like the movie 2012 ? 
Context: ﻿SrNo: 7
MovieName: 2012
Rating: 1
Polarity: 0

Comments: This movie was a complete shocker. The acting was so bad I could not stay with it. When acting is terrible it is

Comments: Truly awful film, with uninspired action, and acting from most of the cast which is just as bad.

Comments: I see that I am not alone, other people were dissapointed with this pathetic,decidely tawdry attempt at a film. The 
Answer: The acting was terribly bad and disappointed viewers found the movie to be a complete shocker, pathetic, and tawdry attempt at a film with uninspired action and awful acting from most of the cast.


In [39]:
rag_chain2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt2
    | llm
    | StrOutputParser()
)

generated_answer2 = rag_chain2.invoke("Why audience didn't like the movie 2012 ?")
print(generated_answer2)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Start answer with apology statement. Always say "thanks for asking!" at the end of the answer. 
﻿SrNo: 7
MovieName: 2012
Rating: 1
Polarity: 0

Comments: This movie was a complete shocker. The acting was so bad I could not stay with it. When acting is terrible it is

Comments: Truly awful film, with uninspired action, and acting from most of the cast which is just as bad.

Comments: I see that I am not alone, other people were dissapointed with this pathetic,decidely tawdry attempt at a film. The
Question: Why audience didn't like the movie 2012 ?
Helpful Answer: We apologize for any disappointment caused by the movie 2012. Based on feedback from our viewers, the acting was terribly poor, the action was uninspired, and overall, the film fell short of expecta